In [1]:
import pandas as pd

In [3]:
cancerDataSet = pd.read_csv("BreastCancerWisconsinDataSet.csv")

In [4]:
y = cancerDataSet['diagnosis'].map(lambda d: 1 if d == 'M' else 0).values
X = cancerDataSet.drop(['diagnosis', 'id'], axis=1).values

In [5]:
# Partition the input data into a training set and a test set
m = X.shape[0]
m_train = 400
m_test = m - m_train

X_train = X[:m_train]
y_train = y[:m_train]

X_test = X[m_train:]
y_test = y[m_train:]

# Gets a batch of the training data. 
# NOTE: Rather than loading a whole large data set as above and then taking array slices as done here, 
#       This method can connect to a data source and select just the batch needed.
def get_training_batch(batch_num, batch_size):
    lower = batch_num * (m_train // batch_size)
    upper = lower + batch_size
    return X_train[lower:upper], y_train[lower:upper]

## TensorFlow Implementation

In [32]:
import tensorflow as tf
import numpy as np

# Make this notebook's output stable across runs
tf.reset_default_graph()
tf.set_random_seed(42)
np.random.seed(42)

# A method to build a new neural net layer of a given size,  
# fully connect it to a given preceding layer X, and 
# compute its output Z either with or without (default) an activation function
# Call with activation=tf.nn.relu or tf.nn.sigmoid or tf.nn.tanh, for examples

def make_nn_layer(layer_name, layer_size, X, activation=None):
    with tf.name_scope(layer_name):
        X_size = int(X.get_shape()[1])
        SD = 2 / np.sqrt(X_size)
        weights = tf.truncated_normal((X_size, layer_size), dtype=tf.float64, stddev=SD)
        W = tf.Variable(weights, name='weights')
        b = tf.Variable(tf.zeros([layer_size], dtype=tf.float64), name='biases')
        Z = tf.matmul(X, W) + b
        if activation is not None:
            return activation(Z)
        else:
            return Z

In [44]:
# Make the neural net structure
n = int(X.shape[1])
print(n)
n_inputs = n
n_hidden1 = 10
n_hidden2 = 5
n_outputs = 2   # Two output classes: malignant or non-malignant

X = tf.placeholder(tf.float64, shape=(None, n_inputs), name='X')

with tf.name_scope('nn'):
    hidden1 = make_nn_layer('hidden1', n_hidden1, X, activation=tf.nn.relu)
    hidden2 = make_nn_layer('hidden2', n_hidden2, hidden1, activation=tf.nn.relu)
    outputs = make_nn_layer('outputs', n_outputs, hidden2) 
    outputs = tf.identity(outputs, "nn_output")
    
y = tf.placeholder(tf.int64, shape=(None), name='y')

30


In [45]:
# Define how the neural net will learn

with tf.name_scope('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=outputs)
    loss = tf.reduce_mean(xentropy, name='l')
    
learning_rate = 0.0001
with tf.name_scope("train"):
#    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    optimizer = tf.train.AdamOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)
    
with tf.name_scope("test"):
    correct = tf.nn.in_top_k(tf.cast(outputs, tf.float32), y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))    

In [46]:
# Set up the ability to save and restore the trained neural net...
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [134]:
# TRAINING TIME

# This is how many times to use the full set of training data
n_epochs = 1000

# For a larger training set, it's typically necessary to break training into
# batches so only the memory needed to store one batch of training data is used
batch_size = 50

with tf.Session() as training_session:
    init.run()
    
    for epoch in range(n_epochs):
        
        # Loop through the whole training set in batches
        for batch_num in range(m_train // batch_size):
            X_batch, y_batch = get_training_batch(batch_num, batch_size)
            training_session.run(training_op, feed_dict={X: X_batch, y: y_batch})

        if epoch % 100 == 99:
            acc_train = accuracy.eval(feed_dict={X: X_train, y: y_train})
            acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test})
            print(epoch+1, "Training accuracy:", acc_train, "Testing accuracy:", acc_test)

    save_path = saver.save(training_session, ".\CancerNN")
    
    # A quick test with the trained model 
    Z = outputs.eval(feed_dict={X: X_test[:40]})
    y_pred = np.argmax(Z, axis=1)
    print("")
    print("Actual classes:   ", y_test[:40])  
    print("Predicted classes:", y_pred)

100 Training accuracy: 0.5675 Testing accuracy: 0.7692308
200 Training accuracy: 0.885 Testing accuracy: 0.70414203
300 Training accuracy: 0.8925 Testing accuracy: 0.7692308
400 Training accuracy: 0.895 Testing accuracy: 0.7692308
500 Training accuracy: 0.9125 Testing accuracy: 0.7751479
600 Training accuracy: 0.9125 Testing accuracy: 0.7751479
700 Training accuracy: 0.9125 Testing accuracy: 0.7810651
800 Training accuracy: 0.9175 Testing accuracy: 0.7810651
900 Training accuracy: 0.92 Testing accuracy: 0.7810651
1000 Training accuracy: 0.92 Testing accuracy: 0.7810651

Actual classes:    [1 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0 1 0
 0 0 0]
Predicted classes: [1 0 0 0 0 0 1 0 1 0 0 0 0 1 1 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 1 0
 0 1 0]


## Keras Implementation

In [155]:
from keras.models import Model
from keras.layers import Activation, Dense, Dropout, Input
from keras.optimizers import Adam, RMSprop
from numpy.random import seed
seed(1)

In [150]:
print(X_train.shape)
print(y_train.shape)
print(X_train[0])
print(y_train[0])
print(X_train[5])
print(y_train[50])

(400, 30)
(400,)
[1.799e+01 1.038e+01 1.228e+02 1.001e+03 1.184e-01 2.776e-01 3.001e-01
 1.471e-01 2.419e-01 7.871e-02 1.095e+00 9.053e-01 8.589e+00 1.534e+02
 6.399e-03 4.904e-02 5.373e-02 1.587e-02 3.003e-02 6.193e-03 2.538e+01
 1.733e+01 1.846e+02 2.019e+03 1.622e-01 6.656e-01 7.119e-01 2.654e-01
 4.601e-01 1.189e-01]
1
[1.245e+01 1.570e+01 8.257e+01 4.771e+02 1.278e-01 1.700e-01 1.578e-01
 8.089e-02 2.087e-01 7.613e-02 3.345e-01 8.902e-01 2.217e+00 2.719e+01
 7.510e-03 3.345e-02 3.672e-02 1.137e-02 2.165e-02 5.082e-03 1.547e+01
 2.375e+01 1.034e+02 7.416e+02 1.791e-01 5.249e-01 5.355e-01 1.741e-01
 3.985e-01 1.244e-01]
0


In [158]:
batch_size = 100

Inputs = Input(shape=(30,))
Dense_1 = Dense(30)(Inputs)
Dense_2 = Dense(30, activation="relu")(Dense_1)
Dense_3 = Dense(30, activation="relu")(Dense_2)
Dense_4 = Dense(2, activation="linear")(Dense_3)
Classifier = Activation('softmax')(Dense_4)

model = Model(input=Inputs, output=Classifier)
model.summary()

#optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, amsgrad=False)
optimizer = RMSprop(lr=0.0001)
    
model.compile(loss='sparse_categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])
 
history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=10,
                    verbose=1,
                    validation_split=0.1)

C:\Users\GUILHAUMELeroy-Melin\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("ac...)`
  # Remove the CWD from sys.path while we load stuff.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_73 (InputLayer)        (None, 30)                0         
_________________________________________________________________
dense_287 (Dense)            (None, 30)                930       
_________________________________________________________________
dense_288 (Dense)            (None, 30)                930       
_________________________________________________________________
dense_289 (Dense)            (None, 30)                930       
_________________________________________________________________
dense_290 (Dense)            (None, 2)                 62        
_________________________________________________________________
activation_24 (Activation)   (None, 2)                 0         
Total params: 2,852
Trainable params: 2,852
Non-trainable params: 0
_________________________________________________________________
Train on

In [160]:
score = model.evaluate(X_test, y_test,
                       batch_size=batch_size, verbose=1)
 
print('Test accuracy:', score[1])

169/169 [==============================] - 0s 71us/step
Test accuracy: 0.62721893032627
